# Distance: Flying Experiments

Detect the distance to the wall while flying and playing sweeps.

In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

#import rclpy
#rclpy.init()

In [ ]:
def get_calib_function(calib="stepper"):
    from utils.calibration import get_calibration_function_median
    from utils.calibration import get_calibration_function_moving
    if calib == "flying":
        print("using flying")
        calib_function_median, freqs = get_calibration_function_moving(
            "2021_10_12_flying", motors="linear_buzzer_cont", fit_one_gain=False,
            #"2021_07_14_flying", motors="linear_buzzer_cont", fit_one_gain=False,
            appendix_list=["_new3", "_new4", "_new6"], 
        )
    elif calib == "stepper":
        print("using stepper dataset")
        calib_function_median, freqs = get_calibration_function_median(
            #"2021_07_08_stepper_fast",
            "2021_10_07_stepper_new_f",
            motors="all45000",
            mic_type="audio_deck",
            snr=5,
            fit_one_gain=False,
        )
    return calib_function_median

def get_estimates_here(results_matrix, xvalues):
    valid = np.any(results_matrix > 0, axis=0)
    estimates = xvalues[np.argmax(results_matrix, axis=0)].astype(float)
    estimates[~valid] = np.nan
    return estimates

In [ ]:
def plot_positions(ax, positions_cm, walls):
    from matplotlib import cm
    
    cmap = cm.get_cmap('inferno') 
    n_labels = 3
    label = None
    step = len(positions_cm) // n_labels
    for i, p in enumerate(positions_cm):
        if i % step == 0 or (i == len(positions_cm) - 1):
            label = f'position {i}'
        ax.scatter(*p[:2], color=cmap(i / len(positions_cm)), label=label)
        label=None
    ax.plot(positions_cm[:, 0], positions_cm[:, 1], color='k', ls=':')
    ax.axis('equal')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend([handles[0], handles[-1]], ["start", "end"], loc='upper right')
    
    for wall_distance, wall_angle in walls:
        print(wall_distance, wall_angle)
        ax.arrow(0, 0, wall_distance * np.cos(wall_angle / 180 * np.pi), 
                 wall_distance * np.sin(wall_angle / 180 * np.pi), color='k', label='_wall', 
                 width=1)
    ax.grid()

## 1. Single wall approach experiments

### Qualitative evaluation

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG
from crazyflie_description_py.parameters import FLYING_HEIGHT_CM
from generate_classifier_results import get_groundtruth_distances, WALLS_DICT

from utils.plotting_tools import FIGSIZE, save_fig

exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new3"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new4"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new6"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new7"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new8"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new10"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_new12"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_1"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_2"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_3"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_4"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_5"
#exp_name="2021_10_12_flying"; motors="linear_buzzer_cont"; appendix="_6"

results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
row = results_df.loc[
    (results_df.appendix == appendix) & (results_df.motors == motors), :
].iloc[0]

bad_freqs = []
n_calib = 5
figsize = 3

flying = row.positions[:, 2] > FLYING_HEIGHT_CM * 1e-2
positions_cm = row.positions[flying, :2] * 1e2
yaws_deg = row.positions[flying, 3]
freqs = row.frequencies_matrix[0, :]
magnitudes = np.abs(row.stft[flying][:, :, freqs>0])
freqs = freqs[freqs > 0]
n_points = magnitudes.shape[0]
times = row.seconds[flying]
distances = get_groundtruth_distances(exp_name, appendix, walls=[[100, 90]], flying=True)

from utils.constants import SPEED_OF_SOUND
print(freqs[1]-freqs[0])
print(SPEED_OF_SOUND)
print(SPEED_OF_SOUND / (4 * (freqs[1]-freqs[0])))

# calibrate and plot calibration
calib_on_the_fly = np.median(magnitudes[:n_calib, :, :], axis=0)
std_values = np.std(magnitudes[:n_calib, :, :], axis=0)
plot_name_calib = f"plots/experiments/{exp_name}{appendix}_on_the_fly.pdf"
fig_calib, axs_calib = plt.subplots(1, 5, sharey=True)
fig_calib.set_size_inches(10, 3)
axs_calib[0].set_ylabel(f"amplitude [-]")
for m in range(4):
    axs_calib[m].plot(freqs, magnitudes[:n_calib, m, :].T, color=f"C{m}", marker='o')
    axs_calib[m].set_title(f"mic{m}")
    axs_calib[4].errorbar(freqs, calib_on_the_fly[m], std_values[m], label=f"mic{m}")
    axs_calib[m].set_xlabel(f"frequency [Hz]")
axs_calib[4].set_xlabel(f"frequency [Hz]")
axs_calib[4].set_title(f"all mics")
axs_calib[0].set_ylim(2, 12)
[axs_calib[0].axvline(f, color='black') for fs in bad_freqs for f in fs]

# plot positions
fig, ax = plt.subplots()
fig.set_size_inches(figsize, figsize)
plot_positions(ax, positions_cm, WALLS_DICT[exp_name])
plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"
save_fig(fig, plot_name)

In [ ]:
import itertools
import time

from utils.plotting_tools import pcolorfast_custom, plot_performance
from utils.inference import Inference
from utils.estimators import DistanceEstimator
from utils.moving_estimators import MovingEstimator
from utils.particle_estimators import ParticleEstimator

# for live analysis
from crazyflie_demo.wall_detection import DISTANCES_CM, ANGLES_DEG
distances_cm = DISTANCES_CM
angles_deg = ANGLES_DEG

# for offline analysis
#distances_cm = np.arange(100, step=1)
#angles_deg = np.arange(360, step=2)

#particle_method = "gaussian"
particle_method = "histogram"
n_calib = 5
n_window = 3
simplify_angles = False

calib_method_list = ["on_the_fly"]#, ["stepper"]
mics_list = [[0, 1, 2, 3]] #, [[1, 3], [0, 2], [0, 1, 3], [0, 1, 2, 3]]
algorithm_list = ["bayes"]#, "cost"]
no_deco = False
errors_df = pd.DataFrame(
    columns=["algorithm", "mics", "calib_method", "estimates", "distances", "appendix", "time"]
)

inf_machine = Inference()
dists_cm = DistanceEstimator.DISTANCES_CM 
azimuths_deg = DistanceEstimator.ANGLES_DEG.astype(float)
inf_machine.add_geometry([dists_cm[0], dists_cm[-1]], WALL_ANGLE_DEG)

for calib_method, mics, algorithm in itertools.product(calib_method_list, mics_list, algorithm_list):
    if calib_method == "on_the_fly":
        calib_values = calib_on_the_fly
    else:
        calib_function = get_calib_function(calib_method)
        calib_values = calib_function(freqs)
    magnitudes_calib = magnitudes / calib_values

    mics_str = str(mics).replace(" ", "")
    plot_name = f"plots/experiments/{exp_name}{appendix}_{algorithm}_{calib_method}_{mics_str}"

    results_matrix = np.full((len(dists_cm), n_points), np.nan)
    results_matrix_angles = np.full((len(azimuths_deg), n_points), np.nan)

    moving_estimator = MovingEstimator(n_window=n_window, distances_cm=distances_cm, angles_deg=angles_deg)
    results_matrix_angles_moving = np.full((len(moving_estimator.angles_deg), n_points), np.nan)
    results_matrix_moving = np.full((len(moving_estimator.distances_cm), n_points), np.nan)
    
    particle_estimator = ParticleEstimator(n_particles=100, global_=False)
    results_matrix_angles_part = np.full((len(particle_estimator.ANGLES_DEG), n_points), np.nan)
    results_matrix_part = np.full((len(particle_estimator.distances_cm), n_points), np.nan)

    time_moving = 0
    time_single = 0
    for i in range(n_points):

        # important to reinitialize!
        distance_estimator = DistanceEstimator()

        inf_machine.add_data(magnitudes_calib[i], freqs)
        inf_machine.filter_out_freqs(freq_ranges=bad_freqs)

        t1 = time.time()
        diff_moving = {}
        for mic_idx in mics:
            dist, prob_mic, diff = inf_machine.do_inference(algorithm, mic_idx)
            distance_estimator.add_distribution(diff * 1e-2, prob_mic, mic_idx)
            diff_moving[mic_idx] = (diff, prob_mic)

        __, prob = distance_estimator.get_distance_distribution(verbose=False, angle_deg=WALL_ANGLE_DEG)
        time_single += int(1000*(time.time() - t1))

        t1 = time.time()
        moving_estimator.add_distributions(
            diff_moving, position_cm=positions_cm[i], rot_deg=yaws_deg[i]
        )
        particle_estimator.add_distributions(
            diff_moving, position_cm=positions_cm[i], rot_deg=yaws_deg[i]
        )
        
        __, prob_moving, __, prob_angle_moving = moving_estimator.get_distributions(simplify_angles=simplify_angles)
        time_moving += int(1000*(time.time() - t1))
        
        __, prob_part, __, prob_angle_part = particle_estimator.get_distributions(simplify_angles=simplify_angles, method=particle_method)
        
        dist_est_cm = dists_cm[np.argmax(prob_moving)]
        __, prob_angle = distance_estimator.get_angle_distribution(
            distance_estimate_cm=dist_est_cm
        )

        results_matrix[:, i] = prob
        results_matrix_angles[:, i] = prob_angle
        results_matrix_moving[:, i] = prob_moving 
        results_matrix_angles_moving[:, i] = prob_angle_moving 
        results_matrix_part[:, i] = prob_part
        results_matrix_angles_part[:, i] = prob_angle_part

        continue
        # DEBUGGING ONLY
        dist, *_ = moving_estimator.get_joint_distribution(simplify_angles=simplify_angles)
        fig, ax = plt.subplots()
        pcolorfast_custom(ax, distances_cm, angles_deg, dist)
        ax.axvline(distances[i], color='white', ls=':')
        ax.axhline(90, color='white', ls=':')
        ax.set_xlabel('distances [cm]')
        ax.set_ylabel('angles [deg]')
        ax.set_title("joint distribution")

    d_estimates = get_estimates_here(results_matrix, dists_cm)
    errors_df.loc[len(errors_df), :] = {
        "algorithm": algorithm,
        "mics": mics_str,
        "calib_method": calib_method,
        "estimates": d_estimates,
        "distances": distances,
        "appendix": appendix,
        "time": time_single / n_points 
    }

    d_estimates_moving = get_estimates_here(results_matrix_moving, distances_cm)
    errors_df.loc[len(errors_df), :] = {
        "algorithm": algorithm + f"_win{n_window}",
        "mics": mics_str,
        "calib_method": calib_method,
        "estimates": d_estimates_moving,
        "distances": distances,
        "appendix": appendix,
        "time": time_moving / n_points 
    }
print("done")

In [ ]:
def plot_matrix(yvalues, results_matrix, gt_values, xvalues=None, no_deco=False, angles=False, log=True, n_calib=0):
    from utils.plotting_tools import add_colorbar, pcolorfast_custom, FIGSIZE
    
    cmap = plt.get_cmap().copy() 
    cmap.set_bad('gray')
    if xvalues is None:
        xvalues = np.arange(results_matrix.shape[1])
        xlabel = "index [-]"
    else:
        xlabel = "time [s]"
        
    estimates = get_estimates_here(results_matrix, yvalues)
    fig, ax = plt.subplots()
    fig.set_size_inches(2 * FIGSIZE, FIGSIZE)
    
    im = ax.pcolormesh(xvalues, yvalues, np.log10(results_matrix) if log else results_matrix, cmap=cmap)
    
    ax.plot(
        xvalues,
        gt_values,
        color="black",
        label="ground truth",
        marker='x'
    )
    ax.set_ylim(min(yvalues), max(yvalues))
    ax.plot(
        xvalues,
        estimates,
        color="white",
        label="estimates",
        marker='o',
        ls='-'
    )
    if no_deco:
        ax.set_yticks([])
        ax.set_xticks([])
    else:
        add_colorbar(fig, ax, im, title="log-probability" if log else "probability")
        ax.set_ylabel("estimated angle [deg]" if angles else "estimated distance [cm]")
        ax.set_xlabel(xlabel)
        
        leg = ax.legend(framealpha=0, loc='upper right')
        for text in leg.get_texts():
            plt.setp(text, color='w')
        #ax.set_xticks(xticks, labels=xticklabels)
        #ax.set_yticks(yticks, labels=yticks)
        #n_xticks = 5
        #step = len(xvalues) // n_xticks
        #ax.set_xticks(np.round(xvalues[::step], 1))
        #ax.set_xticklabels(np.round(xvalues[::step], 1))
        
    alpha = 0.7
    if n_calib > 0:
        from matplotlib.patches import Rectangle
        width = xvalues[n_calib] - xvalues[0]
        height= max(yvalues) - min(yvalues)
        diff_x = xvalues[1] - xvalues[0]
        xy = (min(xvalues) - diff_x/2, min(yvalues))
        rect = Rectangle(xy, width, height, facecolor='white', alpha=alpha)
        ax.add_patch(rect)
        if not no_deco:
            ax.text(xy[0] + width/2, xy[1]+height/2, "used for\ncalibration", fontdict={'color':'black', 'ha':'center', 'va':'bottom'})
    return fig, ax

In [ ]:
fig_size = (6, 4)
fig, ax = plot_matrix(dists_cm, results_matrix, distances, xvalues=times, no_deco=no_deco, 
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix.pdf"
save_fig(fig, plot_name)

fig, ax = plot_matrix(distances_cm, results_matrix_moving, distances, xvalues=times,no_deco=no_deco,
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix_hist.pdf"
save_fig(fig, plot_name)
fig, ax = plot_matrix(particle_estimator.distances_cm, results_matrix_part, distances, xvalues=times, no_deco=no_deco,
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix_part.pdf"
save_fig(fig, plot_name)

gt_angles = np.full(len(distances), 90)
fig, ax = plot_matrix(azimuths_deg, results_matrix_angles, gt_angles, xvalues=times,
                      no_deco=no_deco, angles=True,  
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix_angle.pdf"
save_fig(fig, plot_name)
fig, ax = plot_matrix(np.array(angles_deg), results_matrix_angles_moving,  gt_angles, xvalues=times,
                      no_deco=no_deco, log=False, angles=True,
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix_hist_angle.pdf"
save_fig(fig, plot_name)
fig, ax = plot_matrix(np.array(particle_estimator.ANGLES_DEG), results_matrix_angles_part,  gt_angles, xvalues=times,
                      no_deco=no_deco, log=False, angles=True,
                      n_calib=n_calib)
fig.set_size_inches(*fig_size)
plot_name = f"plots/experiments/{exp_name}{appendix}_matrix_part_angle.pdf"
save_fig(fig, plot_name)

### Factor graph test

In [ ]:
from factor_graph.plot import plot_projections
import gtsam
from audio_gtsam.wall_backend import WallBackend

X = gtsam.symbol_shorthand.X
P = gtsam.symbol_shorthand.P

wall_backend = WallBackend(use_isam=True)
yaw_start = np.pi 

use_groundtruth = False
#use_groundtruth = True 

dist_matrix = results_matrix_moving
angle_matrix = results_matrix_angles_moving
n_estimates = 1
#print(angle_matrix.shape, dist_matrix.shape)

d_estimates = []
d_gt = []

for i, prob_dists in enumerate(dist_matrix.T):
    t1 = time.time()
    if i < n_calib:
        continue
    
    position_cm = np.r_[positions_cm[i, :], 0.0]
    yaw = yaws_deg[i] / 180 * np.pi
    pose_factor = wall_backend.add_pose(r_world=position_cm * 1e-2,  yaw=yaw, verbose=False)
    
    distance_gt = distances[i]
    
    if use_groundtruth:
        prob_dists = np.zeros(len(distances_cm))
        idx = np.where(distances_cm == distance_gt)[0]
        prob_dists[idx] = 1.0
        
        prob_angles = np.zeros(len(angles_deg))
        prob_angles[angles_deg==90] = 1.0
    else:
        prob_angles = angle_matrix[:, i]
        
    wall_backend.add_planes_from_distributions(np.array(distances_cm), prob_dists, 
                                               np.array(angles_deg), prob_angles, 
                                               limit_distance=20,
                                               n_estimates=n_estimates, 
                                               verbose=False)
    
    wall_backend.check_wall(verbose=False)
    distance_estimate = wall_backend.get_distance_estimate()
    d_estimates.append(distance_estimate * 1e2 if distance_estimate else np.nan)
    d_gt.append(distance_gt if distance_estimate else np.nan)
    #print(f"time with isam={wall_backend.use_isam}: {(time.time() - t1)*1e3:.0f}ms")

errors_df.loc[len(errors_df), :] = {
    "algorithm": algorithm + f"_win{n_window}_factorgraph",
    "mics": mics_str,
    "calib_method": calib_method,
    "estimates": d_estimates,
    "distances": d_gt,
    "appendix": appendix,
    "time": time_moving / n_points 
}
   
#plot_projections(wall_backend.all_initial_estimates, axis_length=0.2, ls=":", perspective=False, side=True)
#plt.show()
wall_backend.get_results()
plot_projections(wall_backend.result, axis_length=0.1, perspective=False, side=True)
plt.show()

### quantitative evaluation

In [ ]:
def plot_random(axs, dists):
    np.random.seed(1)
    errors_rand = []
    for d in dists:
        d_rand = np.random.choice(range(7, 100))
        errors_rand.append(d_rand - d)
    axs[0].plot(dists, errors_rand, color='k', label='random', ls='', marker='x', markersize=2)
    axs[1].plot(sorted(np.abs(errors_rand)), np.linspace(0, 1, len(errors_rand)), color='k', ls='', label='random', marker='x', markersize=2)
    axs[1].legend(loc='lower right')

In [ ]:
from utils.plotting_tools import plot_performance, save_fig

groupby = ['mics', 'algorithm']

total_err_dict = {}
for j, (labels, df) in enumerate(errors_df.groupby(groupby, sort=False)):
    params = dict(zip(groupby, labels))
        
    err_dict = {}
    dist_dict = {}
    errors_all = []
    distances_all = []
    for i, row in df.iterrows():
        errors = np.array(row.estimates) - np.array(row.distances)
        key = ' '.join(row.drop(['distances', 'estimates', 'time'] + groupby).values)
        err_dict[key] = errors
        dist_dict[key] = row.distances
        errors_all += list(errors[~np.isnan(errors)])
        distances_all += list(np.array(row.distances)[~np.isnan(errors)])
        
    sort_idx = np.argsort(distances_all)
    total_err_dict[j] = {
        'errors': np.array(errors_all)[sort_idx],
        'distances': np.array(distances_all)[sort_idx],
        **params
    }
    
    title = ''
    for k,l in params.items():
        title = f"{title} {k}: {l}"

    fig, axs = plot_performance(err_dict, xs_dict=dist_dict, xlabel="distance [cm]", ylabel="error [cm]")
    plot_random(axs, range(10, 80))
    #fig, axs = plot_performance(err_dict, xlabel="distance [cm]", ylabel="error [cm]")
    fig.suptitle(title)
    axs[0].get_legend().set_visible(False)
    axs[0].set_ylim(-60, 60)
    axs[1].set_xlim(-2, 60)
    #save_fig(fig, f'plots/experiments/{exp_name}_{chosen_mics}_{params["algorithm"]}_cdf.pdf')

In [ ]:
max_dist = 40
err_dict = {}
dist_dict = {}
for total_dict in total_err_dict.values():
    key = total_dict['algorithm']
    errors = total_dict['errors']
    dists = total_dict['distances']
    err_dict[key] = errors[dists < max_dist]
    dist_dict[key] = dists[dists < max_dist]
    
fig, axs = plot_performance(err_dict, xs_dict=dist_dict, xlabel="distance [cm]", ylabel="error [cm]", marker_flag=False)
axs[0].get_legend().set_visible(False)
plot_random(axs, range(10, max_dist))
fig.suptitle(f'overall performance up to {max_dist}cm')
save_fig(fig, f'plots/experiments/{exp_name}_cdf.pdf')

## 2. Multi-wall approach experiments 

In [ ]:
exp_name = "2021_11_23_demo"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

# glass wall
#appendix = "hover1" # works super well! 
#appendix = "hover3" # looks good! 
#appendix = "hover5" # looks quite good! 
#appendix = "hover6" # looks ok (from here on, velocity is increased)
appendix = "hover8" # works well (glass wall), but only detects one wall

# normal wall
#appendix = "hover9" # works not great
#appendix = "hover10" # works not great
#appendix = "hover11" # works okay
#appendix = "hover12" # works okay 
wall_y_cm = 100

In [ ]:
exp_name = "2022_01_25"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

appendix = "test24"  # works ok
#appendix = "test25"  # doesn't work

wall_y_cm = 80

In [ ]:
exp_name = "2022_01_27_demo"
results_df = pd.read_pickle(f"../datasets/{exp_name}/all_data.pkl")
print("available appendices:", results_df.appendix.values)

#appendix = "test3" # works ok
appendix = "test4"  # works ok

wall_y_cm = 100

In [ ]:
from utils.plotting_tools import FIGSIZE, save_fig, add_colorbar
from crazyflie_demo.wall_detection import FLYING_HEIGHT_CM
from generate_classifier_results import WALLS_DICT

row = results_df.loc[results_df.appendix == appendix,:].iloc[0]

positions_cm = row.positions[:, :3] * 1e2
flying = (positions_cm[:, 2] > FLYING_HEIGHT_CM) & (positions_cm[:, 2] < 100)
print("not flying:", np.where(~flying)[0])
positions_cm = positions_cm[flying, :]
print(positions_cm.shape)
yaws_deg = row.positions[flying, 3]
freqs = row.frequencies_matrix[0, :]
magnitudes = np.abs(row.stft[flying][:, :, freqs>0])
signals_f = row.stft[flying][:, :, freqs>0]
freqs = freqs[freqs > 0]
times = row.seconds[flying]

n_timesteps = magnitudes.shape[0] 

# because wall is at 90 degrees.
# TODO: make this statement more general
distances_wall1 = wall_y_cm - positions_cm[:, 1]
distances_wall2 = 200 - distances_wall1

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
plot_positions(ax, positions_cm[:n_timesteps, :], walls=WALLS_DICT[exp_name])
plot_name = f"plots/experiments/{exp_name}{appendix}_positions.pdf"

In [ ]:
from generate_classifier_results import DIST_THRESH, STD_THRESH, TAIL_THRESH

def plot_distance_matrix(ax, matrix, distances_cm=None):
    from utils.plotting_tools import pcolorfast_custom
    from copy import deepcopy
    
    if distances_cm is None:
        distances_cm = wall_detection.estimator.distances_cm

    cmap = plt.get_cmap("gray").copy()
    alpha = 0.8
    cmap.set_bad((1 - alpha, 1 - alpha, 1 - alpha))
    log_matrix = deepcopy(matrix)
    log_matrix[matrix > 0] = np.log10(matrix[matrix > 0])
    log_matrix[matrix <= 0] = np.nan
    pcolorfast_custom(
        ax,
        np.arange(log_matrix.shape[1]),
        distances_cm,
        log_matrix,
        n_xticks=12,
        n_yticks=4,
        cmap=cmap,
        alpha=alpha,
    )
    ax.set_ylabel("distance [cm]")


def plot_groundtruth(ax, color="white", **kwargs):
    ax.plot(distances_wall1, color=color, **kwargs)
    ax.plot(distances_wall2, color=color, **kwargs)


def plot_distance_estimates(ax, distance_estimates, label=None, **kwargs):
    ax.plot(distance_estimates, label=label, **kwargs)
    ax.set_ylabel("distance [cm]")
    ax.axhline(DIST_THRESH, color="k", ls="--")


def plot_std_estimates(ax, std_estimates, label=None, **kwargs):
    ax.plot(std_estimates, label=label, **kwargs)
    ax.set_ylabel("standard deviation [cm]")
    ax.axhline(STD_THRESH, color="k", ls="--")
    ax.set_yscale("log")
    ax.grid(True, which="both")


def plot_tail(ax, matrix, **kwargs):
    tail = np.log10(np.mean(matrix[-3:, :], axis=0))
    ax.plot(tail, **kwargs)
    ax.set_ylabel("tail probability")
    ax.grid(True)
    ax.axhline(TAIL_THRESH, color="k", ls="--")


def plot_angles(ax, matrix, **kwargs):
    matrix_norm = matrix / np.sum(matrix, axis=0)[None, :]
    for i in range(matrix_norm.shape[0]):
        angle = ANGLES_DEG[i]
        ax.plot(matrix_norm[i, :], label=f"wall at {angle}", **kwargs)
    ax.legend(loc="upper right")
    ax.grid(True)
    ax.set_ylabel("angle probability")
    
def plot_fg_estimates(ax, d_estimates, colors):
    ax.scatter(np.arange(len(d_estimates)), d_estimates, color=colors)

def plot_all():
    fig, axs = plt.subplots(5, 1, sharex=True)
    fig.set_size_inches(20, 30)

    plot_distance_matrix(axs[0], results_matrix_moving)
    ymin, ymax = axs[0].get_ylim()
    plot_groundtruth(axs[0], color="white")
    plot_distance_estimates(axs[0], distance_estimates, label="chosen method")
    
    plot_fg_estimates(axs[0], fg_dist_estimates, fg_colors)
    axs[0].set_ylim(ymin, ymax)
    
    plot_groundtruth(axs[1], color="black")
    plot_distance_estimates(axs[1], distance_estimates, label="chosen method")
    plot_fg_estimates(axs[1], fg_dist_estimates, fg_colors)
    axs[1].set_ylim(5, 100)
    axs[1].grid(True)

    plot_std_estimates(axs[2], std_estimates, label="chosen method")

    plot_tail(axs[3], results_matrix_moving)

    if not WallDetection.SIMPLIFY_ANGLES:
        plot_angles(axs[4], results_matrix_angles)
    return fig, axs

In [ ]:
from audio_gtsam.wall_backend import WallBackend
from utils.moving_estimators import get_estimate
from crazyflie_demo.wall_detection import WallDetection
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

estimation_method = "mean"
estimator = "particle"

no_deco = False
wall_detection = WallDetection(python_only=True, estimator=estimator)
wall_backend = WallBackend(use_isam=True)

fg_dist_estimates = []
fg_colors = []
fg_angle_estimates = []

angles_forward = []

distance_estimates = []
std_estimates = []

results_matrix_angles = np.full((len(wall_detection.estimator.angles_deg), n_timesteps), np.nan)
results_matrix_moving = np.full((len(wall_detection.estimator.distances_cm), n_timesteps), np.nan)

for i in range(n_timesteps):
    res = wall_detection.listener_callback_offline(
        signals_f[i].T, freqs, positions_cm[i], yaws_deg[i], timestamp=times[i]*1e3
    )
    #fig, ax = plt.subplots()
    #ax.pcolorfast(wall_detection.calibration_data[0, :, :])
    
    if i <= WallDetection.N_CALIBRATION:
        fg_dist_estimates.append(None)
        fg_angle_estimates.append(None)
        fg_colors.append("k")
        
        angles_forward.append(None)
        
        distance_estimates.append(None)
        std_estimates.append(None)
        continue
        
    if res is None:
        print("no result yet!")
        
    #plt.plot(freqs, wall_detection.calibration[0, :])
    #plt.plot(freqs, wall_detection.calibration_std[0, :])
        
    #angle_local = wall_detection.estimator.get_local_forward_angle()
    #angles_forward.append(angle_local)
        
    __, __, prob_moving_dist, prob_moving_angle = res
        
    results_matrix_moving[:, i] = prob_moving_dist
    
    d, std = get_estimate(wall_detection.estimator.distances_cm, prob_moving_dist, method=estimation_method)
    distance_estimates.append(d)
    std_estimates.append(std)
    
    if not WallDetection.SIMPLIFY_ANGLES:
        results_matrix_angles[:, i] = prob_moving_angle
    
    yaw = yaws_deg[i] / 180 * np.pi
    pose_factor = wall_backend.add_pose(r_world=positions_cm[i] * 1e-2,  yaw=yaw, verbose=False)
    #wall_backend.add_planes_from_distributions(distances_cm, prob_moving_dist, angles_deg, prob_moving_angle, n_estimates=1, verbose=True)
    
    wall_backend.add_plane_from_distances(wall_detection.estimator.distances_cm, prob_moving_dist, verbose=False, method=estimation_method)
    #wall_backend.check_wall(verbose=True)
    
    d_estimate = wall_backend.get_distance_estimate()
    angle_deg = wall_backend.get_angle_estimate()
    
    fg_dist_estimates.append(d_estimate * 1e2 if d_estimate else None)
    fg_angle_estimates.append(angle_deg)
    fg_colors.append(f"C{wall_backend.plane_index+1}")
#plt.plot(times[:n_timesteps], angles_forward)

In [ ]:
fig, axs = plot_all()
save_fig(fig, f'plots/experiments/{exp_name}{appendix}_distributions.pdf')

In [ ]:
try:
    matrix_df = pd.read_pickle("results/demo_results_matrices.pkl")
    # full results with all paramteres
    #matrix_df = pd.read_pickle("results/DistanceFlying_matrices.pkl")
    
    # results with a bit fewer parameters
    #matrix_df_0 = pd.read_pickle("results/DistanceFlying_matrices_std0.pkl")
    #matrix_df_1 = pd.read_pickle("results/DistanceFlying_matrices_std1.pkl")
    #matrix_df = pd.concat([matrix_df_0, matrix_df_1])
except FileNotFoundError:
    print("Run generate_flying_results.py to generate results.")
matrix_df

In [ ]:
categories = matrix_df.columns.drop(["matrix distances", "matrix angles", "distances_cm", "angles_deg"]).values

def get_title(row, ignore=[]):
    title = ''
    for param_name in row.index:
        if param_name in ignore:
            continue
        if param_name in categories:
            param_value = row[param_name]
            if type(param_value) in (float, np.float64, np.float32):
                param_value = round(param_value, 1)
            title += f'{param_name}: {param_value}, '
    return title
print(get_title(row))

In [ ]:
methods = ["mean", "peak", "max"]
row = matrix_df.iloc[0]
matrix = row['matrix distances'][0]

fig, axs = plt.subplots(4, 1, sharex=True)
fig.set_size_inches(20, 20)

plot_groundtruth(axs[0], color="white")
plot_groundtruth(axs[1], color="black")
plot_tail(axs[3], matrix)

# try different wall detection schemes
for method in methods:
    distance_estimates = []
    std_estimates = []
    
    for i, prob_moving_dist in enumerate(matrix.T):
        
        d, std = get_estimate(row.distances_cm, prob_moving_dist, method=method)
        distance_estimates.append(d)
        std_estimates.append(std)
    print(len(distance_estimates))
        
    plot_distance_estimates(axs[0], distance_estimates, label=method)
    plot_distance_estimates(axs[1], distance_estimates, label=method)
    plot_std_estimates(axs[2], std_estimates, label=method)
    
axs[1].grid(True)
axs[1].set_ylim(5, 100)

#if not WallDetection.SIMPLIFY_ANGLES:
#    plot_angles(axs[4], results_matrix_angles)
plot_distance_matrix(axs[0], matrix, row.distances_cm)

axs[0].legend(loc='upper right')
axs[1].legend(loc='upper right')
#axs[0].set_ylim(ymin, ymax)

axs[0].set_title(get_title(row))

In [ ]:
try:
    scores = pd.read_pickle("results/DistanceFlying_classifier.pkl")
    scores.loc[~scores["mask bad"].isin(["adaptive", "fixed"]), "mask bad"] = "None"
except FileNotFoundError:
    print("File not found. Generate it using generate_flying_results.py")

In [ ]:
def plot_grid(
    df_here, 
    x_name = "calibration param",
    y_name = "auc",
    row_name = None,
    col_name = None,
    color_name = None,
    marker_name = None,
    linestyle_name = None,
):
    
    from matplotlib.lines import Line2D
    marker_list = list(Line2D.markers.keys())[1:]
    linestyle_list = list(Line2D.lineStyles.keys())
    
    groupby = {}

    if color_name is not None:
        colors = {col: f"C{i}" for i, col in enumerate(df_here[color_name].unique())}
        groupby["color"] = color_name
    else:
        colors = ["C0"]
    
    if marker_name is not None:
        markers = {mark: marker_list[i] for i, mark in enumerate(df_here[marker_name].unique())}
        groupby["marker"] = marker_name
    else:
        markers = [""]
    
    if linestyle_name is not None:
        linestyles = {ls: linestyle_list[i] for i,ls in enumerate(df_here[linestyle_name].unique())}
        groupby["ls"] = linestyle_name
    else:
        linestyles = ["-"]
    
    rows = df_here[row_name].unique()
    cols = df_here[col_name].unique()
    fig, axs = plt.subplots(len(rows), len(cols), sharey=True)
    fig.set_size_inches(5 * len(cols), 5 * len(rows))
    for i, row in enumerate(rows):
        for j, col in enumerate(cols):
            df_plot = df_here.loc[(df_here[row_name] == row) & (df_here[col_name] == col)]
            
            for groupby_vals, df_col in df_plot.groupby(list(groupby.values())):
                
                groupby_dict = dict(zip(groupby.keys(), groupby_vals))
                
                # if these elements are not tin the keys, then we access
                # 0th element of corresponding list.
                l = groupby_dict.get("ls", 0)
                c = groupby_dict.get("color", 0)
                m = groupby_dict.get("marker", 0)
                
                axs[i, j].plot(df_col[x_name], df_col[y_name], ls=linestyles[l], marker=markers[m], color=colors[c])
            axs[i, j].grid(True)
    [axs[0, j].set_title(f"{col_name}:\n{col}") for j, col in enumerate(cols)]
    [axs[i, -1].twinx().set_ylabel(f"{row_name}:\n{row}") for i, row in enumerate(rows)]
    [axs[i, -1].twinx().set_yticks([]) for i, row in enumerate(rows)]
    
    if linestyle_name is not None:
        for label, ls in linestyles.items():
            axs[0, -1].plot([], [], ls=ls, label=label, color="C0")
        axs[0, -1].legend(title=linestyle_name)    
    
    if marker_name is not None:
        for label, marker in markers.items():
            axs[1, -1].plot([], [], marker=marker, label=label, color="C0")
        axs[1, -1].legend(title=marker_name)    
        
    if color_name is not None:
        for label, color in colors.items():
            axs[2, -1].plot([], [], color=color, label=label)
        axs[2, -1].legend(title=color_name)    
    return fig, axs

In [ ]:
df_here = scores[scores.method == "distance-mean"]
fig, axs = plot_grid(df_here, 
    row_name = "mask bad",
    col_name = "calibration name",
    x_name = "calibration param",
    y_name = "auc",
    color_name = "n window",
    marker_name = "simplify angles",
    linestyle_name = "relative std"
)

#import seaborn as sns
#g = sns.FacetGrid(df_here, col="calibration name", row="mask bad", hue="n window", margin_titles=True)
#g.map(sns.pointplot, "calibration param", "auc")
#g.add_legend()

In [ ]:
# based on above, choose parameters. 

from generate_classifier_results import get_groundtruth_distances, get_precision_recall, THRESHOLDS_DICT
from utils.pandas_utils import filter_by_dict

chosen_method = "distance-mean"
chosen_dict = {
    "n window": None, # is filled below
    "simplify angles": False,
    "relative std": 0.0,
    "calibration name": "iir",
    "calibration param": 0.3,
    "mask bad": "None"
}

distances_wall = get_groundtruth_distances("2022_01_27_demo", "test4") 
matrix_df.loc[~matrix_df["mask bad"].isin(["adaptive", "fixed"]), "mask bad"] = "None"
matrix_df = matrix_df.apply(pd.to_numeric, errors='ignore', axis=0)
matrix_df["calibration param"] = np.round(matrix_df["calibration param"], 1)

fig, ax = plt.subplots()
delta = 0.05
for i_window, n_window in enumerate([3, 5, 10]):
    chosen_dict["n window"] = n_window
    rows = filter_by_dict(matrix_df, chosen_dict)
    #[print(key, matrix_df[key].unique()) for key in chosen_dict.keys()]
    assert len(rows) == 1, len(rows)
    matrix = rows.iloc[0]["matrix distances"][0]
    precision, recall = get_precision_recall(matrix, distances_wall, method=chosen_method, verbose=False, sort=False) 
    ax.plot(precision, recall, marker='o', label=f"n window: {n_window}")
    thresholds = THRESHOLDS_DICT[chosen_method]
    for t, p, r in zip(thresholds, precision, recall):
        if r < 1: 
            ax.annotate(t, (p + -delta*i_window, r+delta*i_window))
    fig_mat, ax_mat = plt.subplots()
    ax_mat.pcolorfast(np.arange(matrix.shape[1]), DISTANCES_CM, np.log10(matrix))
    ax_mat.plot(DISTANCES_CM[np.argmax(matrix, axis=0)], color='white')
ax.set_xlabel("precision")
ax.set_ylabel("recall")
ax.set_xlim(-0.1, 1.1)
ax.set_ylim(-0.1, 1.1)
ax.grid(True)
plt.tight_layout()
ax.legend(loc='upper left', bbox_to_anchor=[1.0, 1.0])

### Sandbox

In [ ]:
### alternative angle calculation: geometric averaging based on mic level differences

mic_angles = np.array([90, 180, -90, 0])
mics = [0, 1, 2, 3]

total = np.sum(magnitudes[:, :, :]**2, axis=(0, 2))
powers = np.sum(magnitudes[:, mics]**2, axis=2) / total[None, :]

fig = plt.figure()
for i, mic in enumerate(mics): #range(powers.shape[1]):
    plt.plot(range(powers.shape[0]), powers[:, i], label=f"mic{mic}")
plt.legend()
fig.set_size_inches(10, 5)

fig = plt.figure()
norm = powers[:] / np.mean(powers, axis=1)[:, None]
print(norm[0, :])

angle_vectors = np.array([[np.cos(a / 180 * np.pi), np.sin(a / 180 * np.pi)] for a in mic_angles[mics]])
vectors = norm.dot(angle_vectors)
angles = np.arctan2(vectors[:, 1], vectors[:, 0]) * 180 / np.pi
weights = np.linalg.norm(vectors, axis=1)

significant = np.where(weights > 0.2)[0]

fig = plt.figure()
plt.plot(range(powers.shape[0]), angles, label=f"all")
plt.scatter(significant, angles[significant], label=f"significant")
fig.set_size_inches(10, 5)

fig = plt.figure()
plt.plot(range(powers.shape[0]), weights, label=f"mic{mic}")
fig.set_size_inches(10, 5)

In [ ]:
### statistics tests for alternatives to exponentiation in moving average

import scipy.stats
import scipy.signal
import matplotlib.pylab as plt
import numpy as np

scale1 = 1.0
scale2 = 4.0
loc1=0
loc2=3.0
step = 0.1
xmax = 30

n1 = scipy.stats.norm(loc=loc1, scale=scale1)
n2 = scipy.stats.norm(loc=loc2, scale=scale2)

scale_conv = np.sqrt(scale1**2 + scale2**2)
n_conv = scipy.stats.norm(loc=loc1 + loc2, scale=scale_conv)

values = np.arange(-xmax, xmax, step=step)
n1_pdf = n1.pdf(values)
n1_pdf /= np.sum(n1_pdf)
plt.plot(values, n1_pdf, label=f'N1')

n2_pdf = n2.pdf(values)
n2_pdf /= np.sum(n2_pdf)
plt.plot(values, n2_pdf, label=f'N2')

n_conv_pdf = n_conv.pdf(values)
n_conv_pdf /= np.sum(n_conv_pdf)
plt.plot(values, n_conv_pdf, label=f'analytical')

# compute new distribution through exponentiation. 
n2_new = scipy.stats.norm(loc1 + loc2, scale=scale1)
alpha = scale1**2 / (scale1**2 + scale2**2)

n2_approx_pdf = n2_new.pdf(values) ** alpha
n2_approx_pdf /= np.sum(n2_approx_pdf)
plt.plot(values, n2_approx_pdf, label=f'N1 ^ {alpha:.1f}', ls=":", color='C0')

n2_conv = scipy.signal.fftconvolve(n1_pdf, n2_pdf, mode='same')
plt.plot(values, n2_conv, label=f'N1 * N2', ls=":", color='C1')

plt.legend()

multimodal = np.zeros(len(values))
means = [-5, 0, 7.5]
stds = [1, 2, 3]
print(dict(zip(means, stds)))
plt.figure()
for mean, std in zip(means, stds):
    mode = scipy.stats.norm(loc=mean, scale=std)
    multimodal += mode.pdf(values)
    #plt.plot(values, vals, color='C2', ls=':')
multimodal /= np.sum(multimodal)
plt.plot(values, multimodal, color='C0', label='P1')
plt.plot(values, n2_pdf, color='C1', label='N2')
plt.legend(loc='upper right')

plt.figure()
# shift multimodal distribution
for scale in np.linspace(min(stds), max(stds), 3):
    alpha = scale**2 / (scale**2 + scale2**2)
    multimodal_approx_pdf = np.zeros_like(multimodal) 
    shift = int((loc2 - loc1) / step)
    if shift > 0:
        multimodal_approx_pdf[shift:] = multimodal[:int(len(multimodal) - shift)] ** alpha
    else:
        multimodal_approx_pdf[:int(len(multimodal) - shift)] = multimodal[shift:] ** alpha
    multimodal_approx_pdf /= np.sum(multimodal_approx_pdf)
    plt.plot(values, multimodal_approx_pdf, label=f'P1 ^ {alpha:.1f} (std={scale:.1f})', ls="-")

multimodal_conv = scipy.signal.fftconvolve(multimodal, n2_pdf, mode='same')
multimodal_conv /= np.sum(multimodal_conv)
plt.plot(values, multimodal_conv, label=f'P1 * N2', ls="-")
plt.legend(loc='upper right')

import timeit
print("time for convolution: ", end="")
print(round(timeit.timeit(stmt='scipy.signal.fftconvolve(multimodal, n2_pdf, mode="same")', globals=globals(), number=1000), 2))
print("time for exponentiation: ", end="")
print(round(timeit.timeit(stmt='multimodal_approx_pdf[shift:] = multimodal[:int(len(multimodal) - shift)] ** alpha', globals=globals(), number=1000), 2))